In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
import tiktoken

In [3]:
import os
from google.colab import userdata

api_key = userdata.get("open_ai")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


OPENAI_API_KEY has been set!


In [7]:
from langchain.text_splitter import CharacterTextSplitter
txt_file_path = './The_Quantum_Protector.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [8]:
data

[Document(page_content="Chapter 1: The Breakthrough\n\nDr. Eliana Quantum's relentless pursuit of scientific discovery had always set her apart. At the Quantum Dynamics Lab, her latest endeavor—to create a stable quantum processor—promised to be her most groundbreaking work yet. This processor, capable of performing calculations at speeds unfathomable with classical computers, had the potential to solve complex problems ranging from climate modeling to disease analysis.\n\nEliana's dedication often meant long nights alone in the lab, surrounded by the hum of supercooling units and the glow of quantum arrays. It was during one such night, as the clock neared midnight, that her world changed forever. She initiated a high-risk experiment to entangle quantum bits at a scale never before attempted. Suddenly, a power surge overwhelmed the lab's circuits, triggering an explosion that engulfed the room in blinding light.", metadata={'source': './The_Quantum_Protector.txt'}),
 Document(page_con

In [9]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [10]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [11]:
query = "What is this story about?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'This story follows Dr. Eliana Quantum, a scientist dedicated to creating a stable quantum processor. After an experiment goes wrong, she gains the ability to manipulate quantum fields, giving her powers such as teleportation and the ability to pass through solid objects. The story then follows her battles against the Quantum Syndicate, a criminal organization led by Cipher that seeks control over digital life. Eliana uses her newfound abilities to thwart their plans and protect the world.'

In [12]:
query = "Who is Eliana's enemey?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"Dr. Eliana Quantum's enemy in the story is the Quantum Syndicate, led by Cipher."

In [13]:
query = "How does Eliana defeat them?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"The text doesn't provide specific details on how Eliana defeats the Quantum Syndicate. It only mentions that she faced Cipher in a climactic showdown, with the fate of the digital world hanging in the balance."

In [14]:
query = "Who are Eliana's allies?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"In the story, Dr. Eliana Quantum's ally is Aaron Tech, a hacker whose brilliance was matched only by his idealism. He became her closest confidant and partner in her fight against the Syndicate."